In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_path = '/kaggle/input/chocolate-bar-ratings/flavors_of_cacao.csv'
cc = pd.read_csv(data_path)
cc

In [ ]:
import re
for column in cc.columns:
    cc.columns = cc.columns.str.replace(r'\s*\n\s*', ' ', regex=True)
cc.info()

In [ ]:
cc.rename(columns={
    'Company (Maker-if known)': 'Company',
    'Specific Bean Origin or Bar Name': 'BarName',
    'Review Date':'ReviewDate',
    'Cocoa Percent':'CocoaPercent',
    'Company Location':'CompanyLocation',
    'Bean Type':'BeanType',
    'Broad Bean Origin' : 'BroadBeanOrigin',
}, inplace=True)

In [ ]:
cc.info()

In [ ]:
cc.isnull().sum()

In [ ]:
cc['CocoaPercent'] = cc['CocoaPercent'].str.replace('%', '')
cc['CocoaPercent'] = cc['CocoaPercent'].astype(float)
cc['CocoaPercent'].info()

In [ ]:
cc['BeanType'] = (cc['BeanType']
                 .str.strip()  # Menghapus leading/trailing spaces
                 .replace('', np.nan))  # Mengganti string kosong dengan NaN
cc['BeanType'].replace(np.nan, cc['BeanType'].mode()[0])

In [ ]:
cc['ReviewDate'] = pd.to_datetime(cc['ReviewDate'].astype(str), format='%Y', errors='coerce')
cc['ReviewDate'] = cc['ReviewDate'].dt.year

In [ ]:
cc_numerik = cc.select_dtypes(include=np.number)
cc_numerik.info()
cc_object = cc.select_dtypes(include='object')
cc_object.info()

cc = np.get_dummies(cc, cc_object, drop_first=True)